In [1]:
# ps-28
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=a25015ef5d24c12b978ed4cc41c6478d218142c294b6330bd77fe63f9d754332
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [2]:
# import all necessary module
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
# create spark session
spark = SparkSession.builder.master('local').appName('pyspark-day-27').getOrCreate()
spark

1️⃣  Write down the query to fetch Project name assign to more than one Employee
2️⃣  Get employee name, project name order by firstname from "EmployeeDetail" and"ProjectDetail" for those employee which have assigned project already.

Schema And Data:
================
# Input Data:
data = [
 [1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"],
 [2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"],
 [3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"],
 [4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"],
 [5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"],
]

# Create a schema for the DataFrame
schema = StructType([
 StructField("EmployeeID", IntegerType(), True),
 StructField("First_Name", StringType(), True),
 StructField("Last_Name", StringType(), True),
 StructField("Salary", DoubleType(), True),
 StructField("Joining_Date", StringType(), True),
 StructField("Department", StringType(), True),
 StructField("Gender", StringType(), True)
])

pro_schema = StructType([
StructField("Project_DetailID", IntegerType(), True),
StructField("Employee_DetailID", IntegerType(), True),
StructField("Project_Name", StringType(), True)
])

# Create the data as a list of tuples
pro_data = [
(1, 1, "Task Track"),
(2, 1, "CLP"),
(3, 1, "Survey Management"),
(4, 2, "HR Management"),
(5, 3, "Task Track"),
(6, 3, "GRS"),
(7, 3, "DDS"),
(8, 4, "HR Management"),
(9, 6, "GL Management")
]

In [4]:
# Input Data:
data = [
 [1, "Vikas", "Ahlawat", 600000.0, "2013-02-15 11:16:28.290", "IT", "Male"],
 [2, "nikita", "Jain", 530000.0, "2014-01-09 17:31:07.793", "HR", "Female"],
 [3, "Ashish", "Kumar", 1000000.0, "2014-01-09 10:05:07.793", "IT", "Male"],
 [4, "Nikhil", "Sharma", 480000.0, "2014-01-09 09:00:07.793", "HR", "Male"],
 [5, "anish", "kadian", 500000.0, "2014-01-09 09:31:07.793", "Payroll", "Male"],
]

# Create a schema for the DataFrame
schema = StructType([
 StructField("EmployeeID", IntegerType(), True),
 StructField("First_Name", StringType(), True),
 StructField("Last_Name", StringType(), True),
 StructField("Salary", DoubleType(), True),
 StructField("Joining_Date", StringType(), True),
 StructField("Department", StringType(), True),
 StructField("Gender", StringType(), True)
])

In [5]:
emp_df = spark.createDataFrame(schema = schema, data = data)
emp_df.show()

+----------+----------+---------+---------+--------------------+----------+------+
|EmployeeID|First_Name|Last_Name|   Salary|        Joining_Date|Department|Gender|
+----------+----------+---------+---------+--------------------+----------+------+
|         1|     Vikas|  Ahlawat| 600000.0|2013-02-15 11:16:...|        IT|  Male|
|         2|    nikita|     Jain| 530000.0|2014-01-09 17:31:...|        HR|Female|
|         3|    Ashish|    Kumar|1000000.0|2014-01-09 10:05:...|        IT|  Male|
|         4|    Nikhil|   Sharma| 480000.0|2014-01-09 09:00:...|        HR|  Male|
|         5|     anish|   kadian| 500000.0|2014-01-09 09:31:...|   Payroll|  Male|
+----------+----------+---------+---------+--------------------+----------+------+



In [6]:
pro_schema = StructType([
StructField("Project_DetailID", IntegerType(), True),
StructField("Employee_DetailID", IntegerType(), True),
StructField("Project_Name", StringType(), True)
])

# Create the data as a list of tuples
pro_data = [
(1, 1, "Task Track"),
(2, 1, "CLP"),
(3, 1, "Survey Management"),
(4, 2, "HR Management"),
(5, 3, "Task Track"),
(6, 3, "GRS"),
(7, 3, "DDS"),
(8, 4, "HR Management"),
(9, 6, "GL Management")
]

In [7]:
# create df for project
pro_df = spark.createDataFrame(schema = pro_schema, data = pro_data)
pro_df.show()

+----------------+-----------------+-----------------+
|Project_DetailID|Employee_DetailID|     Project_Name|
+----------------+-----------------+-----------------+
|               1|                1|       Task Track|
|               2|                1|              CLP|
|               3|                1|Survey Management|
|               4|                2|    HR Management|
|               5|                3|       Task Track|
|               6|                3|              GRS|
|               7|                3|              DDS|
|               8|                4|    HR Management|
|               9|                6|    GL Management|
+----------------+-----------------+-----------------+



In [8]:
pro_df.groupBy('Project_Name').agg(count('*').alias('count_pro')).filter(col('count_pro') > 1).show()

+-------------+---------+
| Project_Name|count_pro|
+-------------+---------+
|HR Management|        2|
|   Task Track|        2|
+-------------+---------+



In [10]:
emp_df.join(pro_df, emp_df['EmployeeID'] == pro_df['Employee_DetailID'],'Inner')\
      .orderBy(lower(col('First_Name')))\
      .select('First_Name','Project_Name')\
      .show()

+----------+-----------------+
|First_Name|     Project_Name|
+----------+-----------------+
|    Ashish|       Task Track|
|    Ashish|              GRS|
|    Ashish|              DDS|
|    Nikhil|    HR Management|
|    nikita|    HR Management|
|     Vikas|       Task Track|
|     Vikas|              CLP|
|     Vikas|Survey Management|
+----------+-----------------+

